## TODO
* Features:
    * Easy visualization of: case starts, concurrent case obligations
    * printouts/reports of parameters used to generate analyses
    * utils: determine if an event falls in an arbitrary time window; aggregate to show a weekly incidence (across days of week), vs 52 weeks/yr, 12 months/year; expected formats for activity events 
    * some clinical questions to be answered: mishaps during nights and wkends vs weekdays, per provider, or likelihood when activity (anesthesia demand) is high
    * start and end event priorities for defining an activity instance-->needs to eventually create a timeseries (long format) and use these as start and end
    
## Next
* can I re-implement the "reset study" thing?
* multi-threading?
* test cases
    * EventActivityDefinition
        * general instantiation
        * generating ActivityDataSet
        * offsets
    * ActivityDataSet
        * (re-)generate duration
        * enforce maximum duration
* should there be a ConcurrencyTimeSeries object to extend Series and tack on some common functionality?

In [ ]:
import sys
sys.path.append("/home/tj/PycharmProjects/pytiva")

import os
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from tqdm import tqdm

import pytiva

In [ ]:
# keep this
wd = r'/home/tj/PycharmProjects/pytiva/tests/test_data'

csv_dict = {
    'ds_cases': {
        'FILEPATH': os.path.join(wd, 'anesthesia_case_test_data.csv'),
    },

    'ds_case_meds': {
        'FILEPATH': os.path.join(wd, 'med_test_data.csv'),
    },
    'ds_case_events': {
        'FILEPATH': os.path.join(wd, 'event_test_data.csv'),
    }
}

ANESSTART_DATES = ('2021-10-01', '2021-12-31')
LST_PROCEDURES = ['LABOR EPIDURAL/ANALGESIA', 'CESAREAN SECTION PRIMARY 1919']

STUDY_CONFIG = {
    'CASE_LIMITS': {
        'ANESTHESIA_START_RANGE': ANESSTART_DATES,
        'PROCEDURES': LST_PROCEDURES
    }
}

In [ ]:
datasets = pytiva.anesthesia.datasets_from_csv_data(csv_dict)
study = pytiva.anesthesia.AnesthesiaStudy(**datasets)
study._member_ds_and_counts

In [ ]:
datasets = pytiva.anesthesia.datasets_from_csv_data(csv_dict)
study = pytiva.anesthesia.AnesthesiaStudy(**datasets)
study.limit_by_dates(*ANESSTART_DATES)
study._member_ds_and_counts

In [ ]:
datasets = pytiva.anesthesia.datasets_from_csv_data(csv_dict)
study = pytiva.anesthesia.AnesthesiaStudy(**datasets)
study.limit_by_list('procedure', LST_PROCEDURES)
study._member_ds_and_counts

In [ ]:
datasets = pytiva.anesthesia.datasets_from_csv_data(csv_dict)
study = pytiva.anesthesia.AnesthesiaStudy(**datasets)
study.process_study_config(STUDY_CONFIG)
study._member_ds_and_counts

In [23]:
datasets = pytiva.anesthesia.datasets_from_csv_data(csv_dict)
study = pytiva.anesthesia.AnesthesiaStudy(**datasets)
study._member_ds_and_counts

[('ds_cases', 2035), ('ds_case_events', 19296), ('ds_case_meds', 36118)]

In [19]:
datasets = pytiva.anesthesia.datasets_from_csv_data(csv_dict)
study = pytiva.anesthesia.AnesthesiaStudy(**datasets)
study.limit_by_dates(*ANESSTART_DATES)
study._member_ds_and_counts

[('ds_cases', 245), ('ds_case_events', 0), ('ds_case_meds', 0)]

In [21]:
datasets = pytiva.anesthesia.datasets_from_csv_data(csv_dict)
study = pytiva.anesthesia.AnesthesiaStudy(**datasets)
study.limit_by_list('procedure', LST_PROCEDURES)
study._member_ds_and_counts

[('ds_cases', 1580), ('ds_case_events', 0), ('ds_case_meds', 0)]

In [24]:
datasets = pytiva.anesthesia.datasets_from_csv_data(csv_dict)
study = pytiva.anesthesia.AnesthesiaStudy(**datasets)
study.process_study_config(STUDY_CONFIG)
study._member_ds_and_counts

[('ds_cases', 195), ('ds_case_events', 0), ('ds_case_meds', 0)]